### 公众号：数据不吹牛
### 更多案例和有趣分析等你来撩

In [5]:
import pandas as pd
import numpy as np
import os
import json
import requests
import time
import random
from lxml import etree

## 设置基本网址和headers

In [29]:
#两个问题基础网址
gangwei = 'https://www.zhihu.com/api/v4/questions/266817891/answers?include=data%5B%2A%5D.is_normal%2Cadmin_closed_comment%2Creward_info%2Cis_collapsed%2Cannotation_action%2Cannotation_detail%2Ccollapse_reason%2Cis_sticky%2Ccollapsed_by%2Csuggest_edit%2Ccomment_count%2Ccan_comment%2Ccontent%2Ceditable_content%2Cvoteup_count%2Creshipment_settings%2Ccomment_permission%2Ccreated_time%2Cupdated_time%2Creview_info%2Crelevant_info%2Cquestion%2Cexcerpt%2Crelationship.is_authorized%2Cis_author%2Cvoting%2Cis_thanked%2Cis_nothelp%2Cis_labeled%2Cis_recognized%2Cpaid_info%2Cpaid_info_content%3Bdata%5B%2A%5D.mark_infos%5B%2A%5D.url%3Bdata%5B%2A%5D.author.follower_count%2Cbadge%5B%2A%5D.topics&offset={}&limit={}&platform=desktop&sort_by=default'
sanwen = 'https://www.zhihu.com/api/v4/questions/27329739/answers?include=data%5B%2A%5D.is_normal%2Cadmin_closed_comment%2Creward_info%2Cis_collapsed%2Cannotation_action%2Cannotation_detail%2Ccollapse_reason%2Cis_sticky%2Ccollapsed_by%2Csuggest_edit%2Ccomment_count%2Ccan_comment%2Ccontent%2Ceditable_content%2Cvoteup_count%2Creshipment_settings%2Ccomment_permission%2Ccreated_time%2Cupdated_time%2Creview_info%2Crelevant_info%2Cquestion%2Cexcerpt%2Crelationship.is_authorized%2Cis_author%2Cvoting%2Cis_thanked%2Cis_nothelp%2Cis_labeled%2Cis_recognized%2Cpaid_info%2Cpaid_info_content%3Bdata%5B%2A%5D.mark_infos%5B%2A%5D.url%3Bdata%5B%2A%5D.author.follower_count%2Cbadge%5B%2A%5D.topics&offset={}&limit={}&platform=desktop&sort_by=default'

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

## 解析单页信息

In [30]:
def parse_page(url,headers):
    html  = requests.get(url,headers = headers)
    bs = json.loads(html.text)
    result = pd.DataFrame()
    for i in bs['data']:
        headline = i['author']['headline'] #签名
        gender = i['author']['gender']  #性别
        user_type =  i['author']['user_type']
        user_id =  i['author']['id']
        user_token = i['author']['url_token']
        follwer_count = i['author']['follower_count'] #关注人数
        name = i['author']['name']   #用户昵称
        vote_up = i['voteup_count']  #点赞数
        updated_time = i['updated_time']    #更新时间
        title = i['question']['title']   #问题
        created_time = i['created_time'] #创建时间
        comment_count = i['comment_count'] #评论数
        can_comment = i['can_comment']['status']   #是否可以评论
        content = i['content']  #内容，还需要再清洗
        cache = pd.DataFrame({'用户ID':[user_id],'用户名':[name],'性别':[gender],'token':[user_token],'用户类型':[user_type],'签名':[headline],
                              '被关注人数':[follwer_count],'创建时间':[created_time],'更新时间':[updated_time],'评论数':[comment_count],
                              '点赞数':[vote_up],'是否可以评论':[can_comment],'内容':[content],'问题':[title]})
        result = pd.concat([result,cache])
    return result

## 设置爬取回答数，批量获取

In [31]:
def run_all(url,headers,num = 200):
    final_result = pd.DataFrame()
    for i in range(0,num,5):
        try:
            result = parse_page(url.format(i,5),headers)
            final_result = pd.concat([final_result,result])
            time.sleep(random.random())
            print('i had parsed:',i)
        except:
            try:
                time.sleep(5)
                result = parse_page(url.format(i,5),headers)
                final_result = pd.concat([final_result,result])
                time.sleep(random.random())
                print('i had parsed:',i)
            except:
                print(i,'is wrong~~~')  
    return final_result

In [35]:
final_result = run_all(sanwen,headers,775)

i had parsed: 0
i had parsed: 5
i had parsed: 10
i had parsed: 15
i had parsed: 20
i had parsed: 25
i had parsed: 30
i had parsed: 35
i had parsed: 40
i had parsed: 45
i had parsed: 50
i had parsed: 55
i had parsed: 60
i had parsed: 65
i had parsed: 70
i had parsed: 75
i had parsed: 80
i had parsed: 85
i had parsed: 90
i had parsed: 95
i had parsed: 100
i had parsed: 105
i had parsed: 110
i had parsed: 115
i had parsed: 120
i had parsed: 125
i had parsed: 130
i had parsed: 135
i had parsed: 140
i had parsed: 145
i had parsed: 150
i had parsed: 155
i had parsed: 160
i had parsed: 165
i had parsed: 170
i had parsed: 175
i had parsed: 180
i had parsed: 185
i had parsed: 190
i had parsed: 195
i had parsed: 200
i had parsed: 205
i had parsed: 210
i had parsed: 215
i had parsed: 220
i had parsed: 225
i had parsed: 230
i had parsed: 235
i had parsed: 240
i had parsed: 245
i had parsed: 250
i had parsed: 255
i had parsed: 260
i had parsed: 265
i had parsed: 270
i had parsed: 275
i had parsed: 

### 单个爬取用户信息，会存在ip短暂被ban的情况，如果想要持续稳定，建议准备好IP

In [ ]:
def get_ips():
    #交给你了朋友，事先准备好ip，每次调用一个回来就OK

In [20]:
get_ips()

{'https': 'https://117.57.35.166:4512'}

### 逐个爬取用户信息，获取行业岗位等信息

In [1]:
def get_user_info(user,headers,ip):
    user_url = 'https://www.zhihu.com/people/{}/activities'
    ht = requests.get(user_url.format(user),headers = headers,proxies = ip)
    if ht.text.find('安全验证') == -1:
        
        bs = etree.HTML(ht.text)
        try:
            hangye = bs.xpath('//div[@class = "ProfileHeader-infoItem"][1]/text()')
        except:
            hangye = None
        try:
            school = bs.xpath('//div[@class = "ProfileHeader-infoItem"][2]/text()')[0]
        except:
            school = None
        try:
            prof = bs.xpath('//div[@class = "ProfileHeader-infoItem"][2]/text()')[1]
        except:
            prof = None
        df = pd.DataFrame({'token':[user],'行业':[hangye],'教育经历':[school],'专业':[prof]})
        
    else:
        
        ip = get_ips()
        try:
            ht = requests.get(user_url.format(user),headers = headers,proxies = ip)
        except:
            ip = get_ips()
            ht = requests.get(user_url.format(user),headers = headers,proxies = ip)
            bs = etree.HTML(ht.text)
        try:
            hangye = bs.xpath('//div[@class = "ProfileHeader-infoItem"][1]/text()')
        except:
            hangye = None
        try:
            school = bs.xpath('//div[@class = "ProfileHeader-infoItem"][2]/text()')[0]
        except:
            school = None
        try:
            prof = bs.xpath('//div[@class = "ProfileHeader-infoItem"][2]/text()')[1]
        except:
            prof = None
        df = pd.DataFrame({'token':[user],'行业':[hangye],'教育经历':[school],'专业':[prof]})
        print('ip changes')
        
    return df,ip

### 循环爬取用户信息

In [ ]:
ip = get_ips()
ct = 1
user_info2 = pd.DataFrame()
for i in final_result['token']:
    df,ip = get_user_info(i,headers,ip)
    user_info2 = pd.concat([user_info2,df])
    time.sleep(random.random() / 2)
    print('i had parsed:{}'.format(ct))
    ct += 1